In [2]:
# Forked from https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py

# Changed the txt file to Tolsoy's 'War and Peace' part1. Also changed encoding to cp1251 (for russian chars).

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.models import load_model
import numpy as np
import random
import sys

Using Theano backend.
Using gpu device 0: GeForce GTX 660 (CNMeM is disabled, cuDNN not available)


In [3]:
path = get_file('tom1.txt', origin='https://raw.githubusercontent.com/just4art/NLP/master/tom1.txt')
text = open(path).read().lower()

#For local: ## text = open('C:/lstm/tom1.txt').read().lower() 
#For all parts:  # + open('C:/lstm/tom2.txt').read().lower() +open('C:/lstm/tom3.txt').read().lower() + open('C:/lstm/tom4.txt').read().lower()

print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

corpus length: 717446
total chars: 126
nb sequences: 239136
Vectorization...
Build model...


In [ ]:
import warnings
warnings.filterwarnings("ignore")

for iteration in range(0 , 50):
    model = load_model('C:\lstm\model%i.h5' % int(iteration-1))
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)
    model.save('C:\lstm\model%i.h5' % int(iteration))


--------------------------------------------------
Iteration 20
Epoch 1/1
239136/239136 [==============================] - 222s - loss: 1.5153     

Unfortunately my browser 'died' during the learning process so there is not all output shown. This part of code worked about 6-7 hours, and last iteration i started from 20th step. 

In [30]:
#Checking the current model state

start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.2]:   #, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence.decode('cp1251') + '"')
    sys.stdout.write(generated.decode('cp1251'))

    for i in range(500):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        #print(next_char)
        sys.stdout.write(next_char.decode('cp1251'))
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "оворить о твоих же интересах. Я тебе гов"
оворить о твоих же интересах. Я тебе говорите в своей и только на него свои стой с нем своими своими выстрелыми собой при своими и под всех положением и при своей и под него на него остановился в своим поднять на нем с том говорите в него своих под войска и становился в него свои столоном, как будто не могу и поднять с своим под всех солдаты. В своих поднял своим в своим под всех своими тому и под всех положения в том, что она при гостиной стоя в него своими всех и встретившие поле своими и поднимали и под всех столовой с нем поднял в
